In [1]:
import os
import json
import time
import shutil
import datetime

In [2]:
from config import setup_logging
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from Product import Product
from app import add_to_db

from concurrent.futures import ThreadPoolExecutor


# Configure logging
logging = setup_logging()

# Set file paths for product URLs and selectors
PRODUCT_URLS_FILE = 'products/product-urls.txt'
SELECTORS_FILE = 'products/product-selectors.txt'

# Set header information
user_agent = 'Mozilla/5.0 (Linux; Android 11; 100011886A Build/RP1A.200720.011) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.69 Safari/537.36'
sec_ch_ua = '"Google Chrome";v="104", " Not;A Brand";v="105", "Chromium";v="104"'
referer = 'https://www.google.com'
cache_control = 'no-cache'
content_type = 'application/json'

# Set up Firefox WebDriver with headless mode
FIREFOX_LOCATION = os.environ.get('FIREFOX_LOCATION', os.getenv('FIREFOX_LOCATION'))
options = Options()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

def save_urls_to_txt(urls, base_url):
    # Create the 'products' directory if it doesn't exist
    if not os.path.exists("products"):
        os.makedirs("products")

    file_name = os.path.join("products", "product-urls.txt")  # Fallback filename if parsing fails

    # Create a set to store unique URLs
    unique_urls = set()

    # Check if the file already exists and read existing URLs into the set
    try:
        if os.path.isfile(file_name):
            with open(file_name, 'r') as file:
                existing_urls = file.read().splitlines()
                unique_urls.update(existing_urls)
    except FileNotFoundError:
        pass

    # Add new unique URLs to the set
    unique_urls.update(urls)

    # Write the unique URLs to the file
    with open(file_name, 'w') as file:
        file.writelines(f"{url}\n" for url in unique_urls)

    logging.info(f'Product URLs saved to {file_name}')

def extract_product_urls(base_url, selector, next_button_selector=None, max_pages=None, max_products=None):
    driver = webdriver.Firefox(options=options)
    try:
        urls = []
        page_count = 0
        current_url = base_url

        while current_url and (max_pages is None or page_count < max_pages):
            try:
                start_time = time.time()
                driver.get(current_url)

                # Calculate dynamic sleep time based on the website's response time
                response_time = time.time() - start_time
                sleep_time = max(1, min(10, response_time * 2))  # Dynamic sleep time (between 1 and 10 seconds)
                time.sleep(sleep_time)

                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                urls.extend(element.get_attribute("href") for element in elements)

                if max_products and len(urls) >= max_products:
                    urls = urls[:max_products]  # Truncate the list to the specified maximum

                if len(urls) >= max_products:
                    break  # Stop when reaching the maximum number of product URLs

                if next_button_selector:
                    next_button = driver.find_element(By.CSS_SELECTOR, next_button_selector)
                    current_url = next_button.get_attribute("href")
                else:
                    current_url = None

                page_count += 1
            except NoSuchElementException:
                logging.info(f"Selector not found on page: {current_url}")
                break
            except TimeoutException:
                logging.info(f"Page load timed out for URL: {current_url}")
                break

        save_urls_to_txt(urls, base_url)  # Save the URLs to a TXT file

        return urls

    finally:
        driver.quit()

def scrape_single_product(product_url, selectors, download_directory='docs'):
    """
    Scrapes product information from a given URL and downloads datasheets if available.

    Args:
        product_url (str): The URL of the product to scrape.
        selectors (dict): A dictionary of selectors for different attributes of the product.
        download_directory (str, optional): The directory to save downloaded datasheets. Defaults to 'docs'.
        
    Returns:
        dict: A dictionary containing scraped product information.
    """
    options = Options()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    product = Product(driver, product_url, **selectors)
    product.scrape()

    product_info = {}
    for key, selector in selectors.items():
        try:
            if key == "image":
                value = product.get_image_url(selector)
            elif key == "brand":
                value = product.get_brand(selector)
            elif key == "datasheets":
                datasheets = product.get_datasheet_links(selector)
                value = []
                for datasheet_name, datasheet_url in datasheets:
                    # Modify each datasheet link to be a clickable name
                    datasheet_link = f"[{datasheet_name}]({datasheet_url})"
                    value.append(datasheet_link)
                    # Download the datasheet using the download_datasheet method
                    product.download_datasheet(datasheet_url, download_directory)
            else:
                value = product.get_attribute(key)
                # Modify the 'name' key to be a clickable link to the product URL
                if key == "name":
                    value = f"[{value}]({product_url})"
            product_info[key] = value
            logging.info(f"{key.capitalize()}: {value}")
        except Exception as e:
            logging.info(f"Failed to retrieve {key} - {e}")

    driver.quit()

    return product_info

def scrape_multiple_products(urls_and_selectors):
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        results = list(executor.map(lambda x: scrape_single_product(*x), urls_and_selectors))

    return results

def read_selectors_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            selectors = json.load(file)
        return selectors
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return {}

def scrape_products():
    # Read product URLs from a file
    with open(PRODUCT_URLS_FILE, 'r') as file:
        product_urls = [line.strip() for line in file]

    # Read selectors from a file
    selectors = read_selectors_from_file(SELECTORS_FILE)

    if not selectors:
        print("No selectors found. Please check the product_selectors.txt file.")
        return

    urls_and_selectors = [(url, selectors) for url in product_urls]

    # Scrape multiple products
    product_info_list = scrape_multiple_products(urls_and_selectors)

    # Create the 'products' directory if it doesn't exist
    os.makedirs("products", exist_ok=True)

    # Save product information to a markdown file in the 'products' directory
    base_url = product_urls[0].split('//')[1].split('/')[0]  # Extract the domain name
    markdown_file_name = f"products/products-info.md"

    with open(markdown_file_name, 'w') as md_file:
        for product_info in product_info_list:
            md_file.write("## Product Information\n\n")
            for key, value in product_info.items():
                if key == "image" and value:  # Check if the key is "image" and there are image URLs
                    md_file.write(f"**{key.capitalize()}**: \n")  # Start the line with the image key
                    for img_url in value:
                        md_file.write(f"![Image]({img_url})\n")  # Add the image tag
                else:
                    md_file.write(f"**{key.capitalize()}**: {value}\n\n")  # Regular text for other keys

    # Find the product URLs file with the timestamp suffix in the 'uploaded' directory
    uploaded_files = [f for f in os.listdir("uploaded") if f.startswith("product-urls-processed_")]
    if uploaded_files:
        uploaded_file = os.path.join("uploaded", uploaded_files[0])

        # Amend the contents of the product URLs file
        with open(uploaded_file, 'a') as uploaded_content:
            for url in product_urls:
                uploaded_content.write(f"{url}\n")

        # Rename the file with the current time and date suffix
        current_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        renamed_file_name = f"uploaded/product-urls-processed_{current_datetime}.txt"
        os.replace(uploaded_file, renamed_file_name)

    return product_info_list

def print_product_list(product_list):
    # Print or process the list of product information as needed
    for product_info in product_list:
        for key, value in product_info.items():
            logging.info(f"{key.capitalize()}: {value}")

/home/bill/tro-assistant/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Pinecone tro-pacific exists!


In [3]:
if __name__ == '__main__':
    base_url = 'https://www.tro.com.au/enclosures/wall-mount-enclosures/steel-wall-mount-enclosures'
    selector = "a.facets-item-cell-grid-title"
    next_button_selector = ".global-views-pagination-next > a"
    max_pages = 1
    max_products = 5

   

In [4]:
 urls = [
        'https://www.tro.com.au/industrial-electrical/contactors-overloads/thermal-overloads'
    ]

In [5]:
    for base_url in urls:
        product_urls = extract_product_urls(base_url, selector, next_button_selector, max_pages=max_pages, max_products=max_products)
        logging.info(product_urls)

    product_info_list = scrape_products()
    print_product_list(product_info_list)
   

INFO:config:Product URLs saved to products/product-urls.txt
INFO:config:['https://www.tro.com.au/7A-10A-Mini-Contactor-Thermal-Overload-LS-Metasol', 'https://www.tro.com.au/1.0A-1.6A-Mini-Contactor-Thermal-Overload-LS-Metasol', 'https://www.tro.com.au/0.4A-0.63A-Mini-Contactor-Thermal-Overload-LS-Metasol', 'https://www.tro.com.au/0.16A-0.25A-Mini-Contactor-Thermal-Overload-LS-Metasol', 'https://www.tro.com.au/6.0A-9.0A-Mini-Contactor-Thermal-Overload-LS-Metasol']
INFO:config:Name: [ILLUMINATED EMERGENCY STOP, 40MM DIAMETER 1NC 48V](https://www.tro.com.au/Giovenzana-Illuminated-Emergency-Stop-40mm-Diameter-1NC-220V_3)
INFO:config:Price: $27.79
INFO:config:Price w/ tax: $30.57
INFO:config:Sku: K-ILLESTOP01-48
INFO:config:Brand: Giovenzana
INFO:config:Details: Giovenzana Illuminated Emergency Stop, 40mm Diameter 1NC 48V

Kit Includes:

1 x PPFL1P4N (Emergency Stop, 40mm Pushbutton)
1 x PL004001 (1NC Contact Block))
1 x PL0045L48 (LED Lamp 48VAC/DC)
 
INFO:config:Datasheets: ['[ K-ILLESTOP

PDF saved as 'docs/media.nl?id=8662060&c=6505412&h=bGM-BAzZtJoJGsiP40s72PZ6NDX5w0-kTiXQ7Y0OIRkSAzzT&_xt=.pdf'


INFO:config:Name: [RELAY RETAINING CLIP DRI-SERIES](https://www.tro.com.au/Weidmuller-Relay-Retaining-Clip-DRI-Series)
INFO:config:Price: $0.42
INFO:config:Price w/ tax: $0.46
INFO:config:Sku: 7760056352
INFO:config:Brand: Weidmuller
INFO:config:Details: Weidmuller Relay Retaining Clip DRI-Series
INFO:config:Datasheets: ['[ 7760056352 Data Sheet](https://www.tro.com.au/core/media/media.nl?id=4263182&c=6505412&h=KVesl6PD8Jfl15iYj98Mojo5xMmkx3Naqs-Ksh3fEaqBjVGy&_xt=.pdf)']


PDF saved as 'docs/media.nl?id=4263182&c=6505412&h=KVesl6PD8Jfl15iYj98Mojo5xMmkx3Naqs-Ksh3fEaqBjVGy&_xt=.pdf'


INFO:config:Name: [EMERGENCY STOP, 40MM DIAMETER 1NC](https://www.tro.com.au/Giovenzana-Emergency-Stop-40mm-Diameter-1NC)
INFO:config:Price: $18.58
INFO:config:Price w/ tax: $20.44
INFO:config:Sku: K-ESTOP01
INFO:config:Brand: Giovenzana
INFO:config:Details: Giovenzana Emergency Stop, 40mm Diameter 1NC

Kit Includes:

1 x PPFN1R4N (Emergency Stop 40mm)
1 x PL004001 (1NC Contact Block)
INFO:config:Datasheets: ['[ K-ESTOP01 Data Sheet](https://www.tro.com.au/core/media/media.nl?id=8662060&c=6505412&h=bGM-BAzZtJoJGsiP40s72PZ6NDX5w0-kTiXQ7Y0OIRkSAzzT&_xt=.pdf)']


...

INFO:config:Name: [GTK-12M 0.21A 3H | 0.16A-0.25A THERMAL OVERLOAD TO SUIT METAMEC MINI CONTACTORS](https://www.tro.com.au/0.16A-0.25A-Mini-Contactor-Thermal-Overload-LS-Metasol)
INFO:config:Price: $21.35
INFO:config:Price w/ tax: $23.49
INFO:config:Sku: LS-1271003000
INFO:config:Brand: LS Electric
INFO:config:Details: GTK-12M 0.21A 3H | 0.16A-0.25A Thermal Overload to Suit MetaMEC Mini Contactors
INFO:config:Datasheets: ['[ LS Mini Contactor Catalogue](https://www.tro.com.au/core/media/media.nl?id=5680683&c=6505412&h=EEKOklU_KSHIvPzIlXIO-FZkKLNIqhzYou1JDW7vwgnXs57z&_xt=.pdf)']


PDF saved as 'docs/media.nl?id=5680683&c=6505412&h=EEKOklU_KSHIvPzIlXIO-FZkKLNIqhzYou1JDW7vwgnXs57z&_xt=.pdf'


INFO:config:Name: [RELAY RETAINING CLIP 4CO DRL-SERIES, METAL](https://www.tro.com.au/Clip-SLD-F-CLIP-4CO-Hold-Down-Clip)
INFO:config:Price: $0.85
INFO:config:Price w/ tax: $0.93
INFO:config:Sku: 7760056235
INFO:config:Brand: Weidmuller
INFO:config:Details: Weidmuller Relay Retaining Clip 4CO DRL-Series, Metal
INFO:config:Datasheets: ['[ 7760056235 Data Sheet](https://www.tro.com.au/core/media/media.nl?id=4262867&c=6505412&h=nSIlpNhoSpbdiJWx_IPt0dvMoGyVHXOOASjGTaFxBnYAqBKc&_xt=.pdf)']


PDF saved as 'docs/media.nl?id=4262867&c=6505412&h=nSIlpNhoSpbdiJWx_IPt0dvMoGyVHXOOASjGTaFxBnYAqBKc&_xt=.pdf'


INFO:config:Name: [ILLUMINATED EMERGENCY STOP, 40MM DIAMETER 1NC 48V](https://www.tro.com.au/Giovenzana-Illuminated-Emergency-Stop-40mm-Diameter-1NC-220V_3)
INFO:config:Price: $27.79
INFO:config:Price w/ tax: $30.57
INFO:config:Sku: K-ILLESTOP01-48
INFO:config:Brand: Giovenzana
INFO:config:Details: Giovenzana Illuminated Emergency Stop, 40mm Diameter 1NC 48V

Kit Includes:

1 x PPFL1P4N (Emergency Stop, 40mm Pushbutton)
1 x PL004001 (1NC Contact Block))
1 x PL0045L48 (LED Lamp 48VAC/DC)
 
INFO:config:Datasheets: ['[ K-ILLESTOP01-48 Data Sheet](https://www.tro.com.au/core/media/media.nl?id=8662060&c=6505412&h=bGM-BAzZtJoJGsiP40s72PZ6NDX5w0-kTiXQ7Y0OIRkSAzzT&_xt=.pdf)']
INFO:config:Name: [RELAY RETAINING CLIP DRI-SERIES](https://www.tro.com.au/Weidmuller-Relay-Retaining-Clip-DRI-Series)
INFO:config:Price: $0.42
INFO:config:Price w/ tax: $0.46
INFO:config:Sku: 7760056352
INFO:config:Brand: Weidmuller
INFO:config:Details: Weidmuller Relay Retaining Clip DRI-Series
INFO:config:Datasheets: [

In [ ]:
    add_to_db()

INFO:config:Retrieving pdf file data...
INFO:config:Splitting pdf file data into chunks...
INFO:config:Context Chunks: [Document(page_content='®\nwww.ipenclosures.comHygienic Electrical Enclosure\nSTAINLESS STEEL | HYGIENIC | IP66, IP69KIP-H453821\n450H x 380W x 210D\nDatasheet_IP-H453821_Rev1\n© IP Enclosures 2021IP69K hygienic food grade electrical enclosures\nare designed to prevent contamination and are suitable for the food & beverage, pharmaceutical and other industries that require high hygienic standards.\nProtection: Complies with IP66 IK07, IP69K\nStandard: DGUV EN1672-2, EN62208, EN/AS60529,\nISO20653, UL508A EHEDG, RoHS Compliant\nMaterial:\n- Body: 1.5mm Grade 304 Stainless Steel- Door: 1.5mm Grade 304 Stainless Steel- Device Mounting Plate: Galvanised Steel Sheet - Seal: Blue Silicone FDA compliant\nBody: The robust monoblock body with 30°sloping roof\nis fabricated using 1.5mm Grade 304 Stainless Steel. It is designed to be completely smooth and without crevices and with

PDF Text: ### Datasheet ### 
®
www.ipenclosures.comHygienic Electrical Enclosure
STAINLESS STEEL | HYGIENIC | IP66, IP69KIP-H453821
450H x 380W x 210D
Datasheet_IP-H453821_Rev1
© IP Enclosures 2021IP69K hygienic food grade electrical enclosures 
are designed to prevent contamination and are suitable for the food & beverage, pharmaceutical and other industries that require high hygienic standards.
Protection: Complies with IP66 IK07, IP69K
Standard: DGUV EN1672-2, EN62208, EN/AS60529, 
ISO20653, UL508A EHEDG, RoHS Compliant
Material:
- Body: 1.5mm Grade 304 Stainless Steel- Door: 1.5mm Grade 304 Stainless Steel- Device Mounting Plate: Galvanised Steel Sheet - Seal: Blue Silicone FDA compliant
Body: The robust monoblock body with 30°sloping roof 
is fabricated using 1.5mm Grade 304 Stainless Steel. It is designed to be completely smooth and without crevices and without concave angles to facilitate cleaning and avoid deposits. The body is designed to promote self draining for ease of hygi

INFO:config:(1 / 15)
INFO:config:Retrieving pdf file data...
INFO:config:Splitting pdf file data into chunks...
INFO:config:Context Chunks: []
INFO:config:Adding pdf data chunks to Pinecone...


PDF Text: ### Datasheet ### 



INFO:config:(2 / 15)
INFO:config:Retrieving pdf file data...
INFO:config:Splitting pdf file data into chunks...
INFO:config:Context Chunks: [Document(page_content='29\n28\nwww.giovenzana.comAutomation Technologies\nwww.giovenzana.com35\n34\nBLGL\nCOLOUR\nINDEXCODEWEIGHT\ngramsMIN. PACK\nQUANTITY\n•\n•••••PPRN1GL\nPPRN2GL\nPPRN3GL\nPPRN4GL\nPPRN5GL\nPPRN8GL303030303030101010101010\nFLUSH\nPUSH BUTTONTYPE\nCOLOUR\nINDEXCODEWEIGHT\ngramsMIN. PACK\nQUANTITY\n•\n••PPFN1M4NGL\nPPFN2M4NGL\nPPFN3M4NGL454545101010\nMUSHROOM\nø40TYPECOLOUR\nINDEXCODEWEIGHT\ngramsMIN. PACK\nQUANTITY\n•\n•••••PPSN1GL\nPPSN2GL\nPPSN3GL\nPPSN4GL\nPPSN5GL\nPPSN8GL313131313131101010101010\nEXTENDED\nPUSH\nBUTTONTYPEAUXILIARY CONTROLS | ORION SERIES | GREY LINE AUXILIARY CONTROLS | ORION SERIES | GREY LINE\nPUSH BUTTONS Ø22 | WITH COLOUR CAP | IP67-IP69K\nSIZES\nSIZESMOMENTARY\nSIZES\nPUSH BUTTONS Ø22 | WITH LENS CAP | IP67-IP69K\nCOLOUR\nINDEXCODEWEIGHT\ngramsMIN. PACK\nQUANTITY\n•\n••••PPRL1GL\nPPRL2GL\nPPRL3GL\nPPRL

...

INFO:config:(3 / 15)
INFO:config:Retrieving pdf file data...
INFO:config:Splitting pdf file data into chunks...
INFO:config:Context Chunks: [Document(page_content='Data sheet  \nCreation date April 10, 2021 12:01:02 PM CEST\nCatalogue status 12.03.2021 / We reserve the right to make technical changes. 1DRM/DRL CLIP MWeidmüller Interface GmbH & Co. KG\nKlingenbergstraße 26\nD-32758 Detmold\nGermany  \nwww.weidmueller.com\nProduct image  \nRetaining clip for D-SERIES DRM relays\n•Metal and plastic retaining clip\nGeneral ordering data  \nVersion D-SERIES, Retaining clip\nOrder No. 7760056108\nType DRM/DRL CLIP M\nGTIN (EAN) 4032248855568\nQty. 10 pc(s).\nData sheet  \nDRM/DRL CLIP MWeidmüller Interface GmbH & Co. KG\nKlingenbergstraße 26\nD-32758 Detmold\nGermany  \nwww.weidmueller.com\nTechnical data  \nCreation date April 10, 2021 12:01:02 PM CEST\nCatalogue status 12.03.2021 / We reserve the right to make technical changes. 2Dimensions and weights  \nNet weight 0.265 g  \nEnvironmenta

PDF Text: ### Datasheet ### 
Data sheet
 
 
Creation date April 10, 2021 12:01:02 PM CEST
  Catalogue status 12.03.2021 / We reserve the right to make technical changes. 1DRM/DRL CLIP MWeidmüller Interface GmbH & Co. KG
Klingenbergstraße 26
D-32758 Detmold
Germany
 
www.weidmueller.com
Product image
 
 
Retaining clip for D-SERIES DRM relays
•Metal and plastic retaining clip
General ordering data
 
Version D-SERIES, Retaining clip
Order No. 7760056108
Type DRM/DRL CLIP M
GTIN (EAN) 4032248855568
Qty. 10 pc(s).
 Data sheet
 
DRM/DRL CLIP MWeidmüller Interface GmbH & Co. KG
Klingenbergstraße 26
D-32758 Detmold
Germany
 
www.weidmueller.com
Technical data
 
 
Creation date April 10, 2021 12:01:02 PM CEST
  Catalogue status 12.03.2021 / We reserve the right to make technical changes. 2Dimensions and weights
 
Net weight 0.265 g   
 
Environmental Product Compliance
 
REACH SVHC Lead 7439-92-1   
 
General data
 
Material metal  Colour silver
 
Classifications
 
ETIM 6.0 EC002586  ETIM 7.0 

INFO:config:(4 / 15)
INFO:config:Retrieving pdf file data...
INFO:config:Splitting pdf file data into chunks...
INFO:config:Context Chunks: [Document(page_content='Data sheet  \nCreation date April 10, 2021 12:53:07 PM CEST\nCatalogue status 12.03.2021 / We reserve the right to make technical changes. 1SDI CLIPWeidmüller Interface GmbH & Co. KG\nKlingenbergstraße 26\nD-32758 Detmold\nGermany  \nwww.weidmueller.com\nProduct image  \nRetaining clip for D-SERIES DRI relays\n•Metal and plastic retaining clip\nGeneral ordering data  \nVersion D-SERIES DRI, Retaining clip\nOrder No. 7760056352\nType SDI CLIP\nGTIN (EAN) 6944169739996\nQty. 10 pc(s).\nData sheet  \nSDI CLIPWeidmüller Interface GmbH & Co. KG\nKlingenbergstraße 26\nD-32758 Detmold\nGermany  \nwww.weidmueller.com\nTechnical data  \nCreation date April 10, 2021 12:53:07 PM CEST\nCatalogue status 12.03.2021 / We reserve the right to make technical changes. 2Dimensions and weights  \nDepth 39.6 mm \xa0Depth (inches) 1.559 inch\nHei

PDF Text: ### Datasheet ### 
Data sheet
 
 
Creation date April 10, 2021 12:53:07 PM CEST
  Catalogue status 12.03.2021 / We reserve the right to make technical changes. 1SDI CLIPWeidmüller Interface GmbH & Co. KG
Klingenbergstraße 26
D-32758 Detmold
Germany
 
www.weidmueller.com
Product image
 
 
Retaining clip for D-SERIES DRI relays
•Metal and plastic retaining clip
General ordering data
 
Version D-SERIES DRI, Retaining clip
Order No. 7760056352
Type SDI CLIP
GTIN (EAN) 6944169739996
Qty. 10 pc(s).
 Data sheet
 
SDI CLIPWeidmüller Interface GmbH & Co. KG
Klingenbergstraße 26
D-32758 Detmold
Germany
 
www.weidmueller.com
Technical data
 
 
Creation date April 10, 2021 12:53:07 PM CEST
  Catalogue status 12.03.2021 / We reserve the right to make technical changes. 2Dimensions and weights
 
Depth 39.6 mm  Depth (inches) 1.559 inch
Height 31 mm  Height (inches) 1.22 inch
Net weight 1.71 g  Width 15.8 mm
Width (inches) 0.622 inch   
 
Temperatures
 
Humidity 35...85 % rel. humidity, no


INFO:config:(5 / 15)
INFO:config:Retrieving pdf file data...
